In [ ]:
import os
import re
import numpy as np
import xarray as xr
import itertools
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Choose chemical mechanism
model_name = 'MCM_C1_C2'
exp_name = 'init'
MCM_RHC1C5 = True
CheT = False

In [ ]:
# Read file with chemical equations (e.g., MCM subset [current setup])
if MCM_RHC1C5:    
    with open('./chemech/MCM_C1_C2.eqn', 'r') as f:
        lines = f.read().splitlines()
    # Read data
    ts = xr.open_dataset(os.path.join(os.path.split(os.getcwd())[0], 'processed', model_name+'_'+exp_name+'.nc'))
    fl = xr.open_dataset(os.path.join(os.path.split(os.getcwd())[0], 'processed', 'flux', 'flux_'+model_name+'_'+exp_name+'.nc'))
if CheT:
    with open('./chemech/diff_CheT_CheTMN_new_and_replaced_reactions.kpp', 'r') as f:
        lines = f.read().splitlines()[1:]

In [ ]:
# Extract reactants, products and reaction rate coefficients from equations
eqs = []
if MCM_RHC1C5:
    for line in lines:
        if not line.startswith('#') and not line.startswith('{{'):
            subline = re.search(r'\t\s(.*)\s:\s\t(.*)\s\t', line).group(1,2)
            reac, prod = [re.findall(r'\w+', s) for s in subline[0].split('=')] # split first tuple value
            eqs.append(dict(reac=reac, prod=prod, coef=subline[1]))
# if CheT:
#     for line in lines:
#         if not line.startswith('{'):
#             subline = re.search(r'\t(.*):\t(.*);', line).group(1,2)
#             reac, prod = [re.findall(r'[A-Za-z]+[\w]*', s) for s in subline[0].split('=')]
#             eqs.append(dict(reac=reac, prod=prod, coef=subline[1]))

In [ ]:
# 'Convert' equations to nodes, edges and edge labels needed for constructing a network
nodes = []
links = [] # connections between reactants and products, edges for networkx
link_labels = {} # 'arrow labels', edge labels for networkx
if MCM_RHC1C5:
    major_reactants = ['CL', 'HO2', 'NO', 'NO2', 'NO3', 'OH', 'SO2', 'SO3'] # 'arrows', no nodes created
elif CheT:
    major_reactants = ['CL', 'CO2', 'H2O', 'HO2', 'M', 'N2', 'NO', 'NO2', 'NO3', 'O2', 'OH', 'SO2', 'SO3']
major_reactants_combos = list(itertools.product(major_reactants, repeat=2))
for eq in eqs:
    for prod in eq['prod']:
        if prod not in major_reactants:
            if len(eq['reac']) == 1 and eq['reac'][0] not in major_reactants: # photolysis
                print(eq)
                link = (eq['reac'][0], prod)
                try:
                    link_labels[link] +=',{}'.format('hv')
                except KeyError:
                    link_labels[link] = 'hv'
                links.append(link)
            elif len(eq['reac']) == 2:
                reac1, reac2 = eq['reac']
                if (reac1, reac2) not in major_reactants_combos: # skip reactions like NO + NO2 = products
                    if reac1 in major_reactants:
                        reac1, reac2 = reac2, reac1 # place reactant like CH3O2 to reac1, major reactant to reac2
                    link = (reac1, prod)
                    links.append(link)
                    try:
                        link_labels[link] +=',{}'.format(reac2)
                    except KeyError:
                        link_labels[link] = reac2 # use major reactant as a link label
                if len(eq['prod']) == 1:
                    if eq['prod'][0] not in nodes:
                        nodes.append(eq['prod'][0]) # add nodes like N2O5 in NO2 + NO3 = N2O5
        else:
            if len(eq['reac']) == 1 and eq['reac'][0] not in major_reactants:
                if eq['reac'][0] not in nodes:
                    nodes.append(eq['reac'][0]) # add nodes like HONO in HONO = OH + NO
            elif len(eq['reac']) == 2:
                reac3, reac4 = eq['reac']
                if (reac3, reac4) not in major_reactants_combos:
                    if reac3 in major_reactants:
                        reac3, reac4 = reac4, reac3
                    if reac3 not in nodes:
                        nodes.append(reac3) # add nodes like CO in OH + CO = HO2

In [ ]:
# Orig
# 'Convert' equations to nodes, edges and edge labels needed for constructing a network
nodes = []
links = [] # connections between reactants and products, edges for networkx
link_labels = {} # 'arrow labels', edge labels for networkx
if MCM_RHC1C5:
    major_reactants = ['CL', 'HO2', 'NO', 'NO2', 'NO3', 'OH', 'SO2', 'SO3'] # 'arrows', no nodes created
elif CheT:
    major_reactants = ['CL', 'CO2', 'H2O', 'HO2', 'M', 'N2', 'NO', 'NO2', 'NO3', 'O2', 'OH', 'SO2', 'SO3']
major_reactants_combos = list(itertools.product(major_reactants, repeat=2))
for eq in eqs:
    for prod in eq['prod']:
        if prod not in major_reactants:
            if len(eq['reac']) == 1 and eq['reac'][0] not in major_reactants: # photolysis
                link = (eq['reac'][0], prod)
                try:
                    link_labels[link] +=',{}'.format('hv')
                except KeyError:
                    link_labels[link] = 'hv'
                links.append(link)
            elif len(eq['reac']) == 2:
                reac1, reac2 = eq['reac']
                if (reac1, reac2) not in major_reactants_combos: # skip reactions like NO + NO2 = products
                    if reac1 in major_reactants:
                        reac1, reac2 = reac2, reac1 # place reactant like CH3O2 to reac1, major reactant to reac2
                    link = (reac1, prod)
                    links.append(link)
                    try:
                        link_labels[link] +=',{}'.format(reac2)
                    except KeyError:
                        link_labels[link] = reac2 # use major reactant as a link label
                if len(eq['prod']) == 1:
                    if eq['prod'][0] not in nodes:
                        nodes.append(eq['prod'][0]) # add nodes like N2O5 in NO2 + NO3 = N2O5
        else:
            if len(eq['reac']) == 1 and eq['reac'][0] not in major_reactants:
                if eq['reac'][0] not in nodes:
                    nodes.append(eq['reac'][0]) # add nodes like HONO in HONO = OH + NO
            elif len(eq['reac']) == 2:
                reac3, reac4 = eq['reac']
                if (reac3, reac4) not in major_reactants_combos:
                    if reac3 in major_reactants:
                        reac3, reac4 = reac4, reac3
                    if reac3 not in nodes:
                        nodes.append(reac3) # add nodes like CO in OH + CO = HO2

In [ ]:
# Create network layout
scheme = nx.MultiDiGraph()
scheme.add_edges_from(links)
scheme.add_nodes_from(nodes)
pos = nx.nx_pydot.graphviz_layout(scheme)

In [ ]:
# Select species to highlight
if MCM_RHC1C5:
    selected_spcs = []
# if MCM_RHC1C5:
#     selected_spcs = ['CH3NO3', 'C2H5NO3', 'IC3H7NO3', 'NC3H7NO3', 'NC4H9NO3', 'SC4H9NO3', 'IC4H9NO3', 'TC4H9NO3', 
#                  'PEANO3', 'PEBNO3', 'PECNO3', 'IPEANO3', 'IPEBNO3', 'IPECNO3']
if CheT:
#     selected_spcs = []
#     selected_spcs = ['MeONO2']
#     selected_spcs = ['MeONO2', 'iPrONO2', 'nPrONO2', 'sBuONO2', 'nPeBCONO2', 'iPeBONO2', 'iPeCONO2']
    selected_spcs = ['iPrONO2', 'nPrONO2', 
                     'MEK', 'nC4H10', 'sBuONO2', 'sBuOO', 'sBuOOH', 
                     'iC4H10', 'iBuOO', 'iBuOOH', 'tBuOO', 'tBuOOH', 'iPrCHO', 'iPrCO3', 
                     'nC5H12', 'nPeBOO', 'nPeCOO', 'nPeBCONO2', 'MPRK', 'DIEK', 'nPeBOOH', 'nPeCOOH', 'DIEKAOO',
                     'DIEKAOOH', 'CO23C5', 'CARB11A', 
                     'iC5H12', 'iPeBOO', 'iPeCOO', 'iPeBONO2', 'iPeCONO2', 'iPeBOOH', 'iPeCOOH', 'MIPK']

In [ ]:
# Filter first-order precursors and products of selected species
selected_spcs_and_depends = []
for n, m in scheme.edges():
    if n in selected_spcs or m in selected_spcs:
        selected_spcs_and_depends.append(n)
        selected_spcs_and_depends.append(m)
selected_spcs_and_depends = set(selected_spcs_and_depends)
# Filter edge labels for production and destruction of selected species
selected_spcs_labels = {k: v for k, v in link_labels.items() if any([i in selected_spcs for i in k])}
# Create subgraph with selected species highlighted
SG=scheme.subgraph(selected_spcs_and_depends)

In [ ]:
# Draw main network
for i in np.arange(20, 40, 10):
    fig, ax = plt.subplots(figsize=(20,20), facecolor='white')
    nx.draw_networkx_nodes(scheme, pos, ax=ax, node_size=i, node_color='grey', alpha=0.3)
    nx.draw_networkx_labels(scheme, pos, ax=ax, font_size=9)
    nx.draw_networkx_edges(scheme, pos, ax=ax, width=0.5, edge_color='grey')
    nx.draw_networkx_edge_labels(scheme, pos, edge_labels=link_labels, font_size=7)
    _ = ax.axis('off')
    ax.set_xlim(-300, 350)
    ax.set_ylim(-300, 350)
    plt.title('CheT+MN: highlight new species and reactions from MN chemistry', fontsize=20) # CheT: highlight MN chemistry

In [ ]:
# Orig
# Draw main network
fig, ax = plt.subplots(figsize=(20,20), facecolor='white')
nx.draw_networkx_nodes(scheme, pos, ax=ax, node_size=30, node_color='grey', alpha=0.3)
nx.draw_networkx_labels(scheme, pos, ax=ax, font_size=9)
nx.draw_networkx_edges(scheme, pos, ax=ax, width=0.5, edge_color='grey')
nx.draw_networkx_edge_labels(scheme, pos, edge_labels=link_labels, font_size=7)
# Draw subnetwork on top of the main network
nx.draw_networkx_nodes(SG, pos, ax=ax, node_size=30, node_color='r', alpha=0.3) # r
nx.draw_networkx_labels(SG, pos, ax=ax, font_size=9, font_color='r') # k
# nx.draw_networkx_edges(SG, pos, ax=ax, width=1.0, edge_color='r') # r
# nx.draw_networkx_edge_labels(SG, pos, edge_labels=selected_spcs_labels, font_size=7)
_ = ax.axis('off')
ax.set_xlim(-300, 350)
ax.set_ylim(-300, 350)
plt.title('CheT+MN: highlight new species and reactions from MN chemistry', fontsize=20) # CheT: highlight MN chemistry

In [ ]:
# Save network
if MCM_RHC1C5:
    fig.savefig('/local/mwe14avu/UEA/PhD/results/plot_network/MCM_RHC1C5_allRONO2marked.svg', format='svg', dpi=100, bbox_inches='tight')
if CheT:
    fig.savefig('/local/mwe14avu/UEA/PhD/results/plot_network/CheT_smth.svg', format='svg', dpi=100, bbox_inches='tight')

In [ ]:
# TODO: find a way to highlight list of nodes and endges without constructing subnetwork